Changes: core->_SoftmaxTfModel -> SoftmaxTfModel
 metrics.reset_states() -> metrics.reset_state()
 remove 0 from the dataset

In [22]:
import tensorflow as tf
import ltn
import baselines, data
from examples import commons
import matplotlib.pyplot as plt
import numpy as np

# Dataset preprocessing
## Importing dataset

In [23]:
mnist = tf.keras.datasets.mnist
(img_train, label_train), (img_test, label_test) = mnist.load_data()

# normalising the pixel values
img_train, img_test = img_train/255.0, img_test/255.0

# adding a channel dimension for compatibility with the convolutional layers
img_train = img_train[...,tf.newaxis]
img_test = img_test[...,tf.newaxis]

## Removing images with the 0 digit

In [24]:
# train data without label 0
not_zeros_train = label_train != 0
img_train = img_train[not_zeros_train]
label_train = label_train[not_zeros_train]

#test data without label 0
not_zeros_test = label_test != 0
img_test = img_test[not_zeros_test]
label_test = label_test[not_zeros_test]

# how much data will be considered
count_train = 10000
count_test = 3000
n_operands = 2

# operation
op = lambda args: args[0]%args[1]

# train data
img_per_operand_train = [img_train[i*count_train:i*count_train+count_train] for i in range(n_operands)]
label_per_operand_train = [label_train[i*count_train:i*count_train+count_train] for i in range(n_operands)]
label_result_train = np.apply_along_axis(op,0,label_per_operand_train)

# test data
img_per_operand_test = [img_test[i*count_test:i*count_test+count_test] for i in range(n_operands)]
label_per_operand_test = [label_test[i*count_test:i*count_test+count_test] for i in range(n_operands)]
label_result_test = np.apply_along_axis(op,0,label_per_operand_test)

## Creating tf datasets of specific buffer and batch size

In [25]:
buffer_size = 3000
batch_size  = 16

# training set
ds_train = tf.data.Dataset.from_tensor_slices(
              ((img_per_operand_train[0],
                img_per_operand_train[1]),
               label_result_train)
           )\
           .shuffle(buffer_size)\
           .batch(batch_size)\
           .prefetch(tf.data.AUTOTUNE)

# test set
ds_test  = tf.data.Dataset.from_tensor_slices(
              ((img_per_operand_test[0],
                img_per_operand_test[1]),
               label_result_test)
           )\
           .batch(batch_size)\
           .prefetch(tf.data.AUTOTUNE)

## Neural Network

In [26]:
# Instantiate model
n_classes = 9
model = baselines.MultiDigits(n_classes=n_classes, hidden_dense_sizes=(84,))

# Compile
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Train
history = model.fit(ds_train, epochs=10)
test_loss, test_accuracy = model.evaluate(ds_test)
print(f"Test accuracy: {test_accuracy:.4f}")
